In [93]:
from selfdrive.car.toyota.values import CAR, EV_HYBRID_CAR, FW_VERSIONS
from selfdrive.car.interfaces import get_torque_params
from deepdiff import DeepDiff
import dictdiffer
from cereal import car

Ecu = car.CarParams.Ecu
ECU_NAME = {v: k for k, v in Ecu.schema.enumerants.items()}

In [27]:
CARs = list(map(str, CAR))
hybrid_map = {'TOYOTA ALPHARD HYBRID 2021': 'TOYOTA ALPHARD 2020',
              'TOYOTA C-HR HYBRID 2022': 'TOYOTA C-HR 2021',
              'TOYOTA HIGHLANDER HYBRID 2018': 'TOYOTA HIGHLANDER 2017',
              # 'LEXUS CT HYBRID 2018': '',
              'LEXUS RX HYBRID 2017': 'LEXUS RX 2016'}
for candidate in CARs:
  if 'HYBRID' not in candidate or candidate in hybrid_map:
    continue
  hybridless = candidate.replace(' HYBRID', '')
  if hybridless in CARs:
    # print(candidate, '->', hybridless)
    hybrid_map[candidate] = hybridless
  else:
    print('?', candidate)
  # print(candidate.replace(' HYBRID', ''))
  

? LEXUS CT HYBRID 2018


In [28]:
assert EV_HYBRID_CAR - {CAR.MIRAI, CAR.PRIUS, CAR.PRIUS_V, CAR.PRIUS_TSS2, CAR.LEXUS_CTH} == set(hybrid_map)

In [82]:
print(len(CAR), len(hybrid_map))
for hybrid_candiate, ice_candidate in hybrid_map.items():
  # print(hybrid_candiate, ice_candidate)
  tph = get_torque_params(hybrid_candiate)
  tpi = get_torque_params(ice_candidate)
  # dd = DeepDiff(tph, tpi, math_epsilon=0.1)
  dd = list(dictdiffer.diff(tph, tpi, tolerance=0, absolute_tolerance=0.1))
  if dd:
    factor_perc_diff = (tph["LAT_ACCEL_FACTOR"] - tpi["LAT_ACCEL_FACTOR"]) / tpi["LAT_ACCEL_FACTOR"] * 100
    maxlat_perc_diff = (tph["MAX_LAT_ACCEL_MEASURED"] - tpi["MAX_LAT_ACCEL_MEASURED"]) / tpi["MAX_LAT_ACCEL_MEASURED"] * 100
    friction_perc_diff = (tph["FRICTION"] - tpi["FRICTION"]) / tpi["FRICTION"] * 100
    print(f'{ice_candidate:>24} -> {hybrid_candiate}')
    print('Factor: {:>16.3f} -> {:.3f} ({:.2f}% change)'.format(
      tpi["LAT_ACCEL_FACTOR"], tph["LAT_ACCEL_FACTOR"], factor_perc_diff
    ))
    print('Measured: {:>14.3f} -> {:.3f} ({:.2f}% change)'.format(
      tpi["MAX_LAT_ACCEL_MEASURED"], tph["MAX_LAT_ACCEL_MEASURED"], maxlat_perc_diff
    ))
    print('Friction: {:>14.3f} -> {:.3f} ({:.2f}% change)'.format(
      tpi["FRICTION"], tph["FRICTION"], friction_perc_diff
    ))
    # print(hybrid_candiate, ice_candidate, dd)
    print()
  
  # print(tph, tpi)
  

51 20
  TOYOTA HIGHLANDER 2017 -> TOYOTA HIGHLANDER HYBRID 2018
Factor:            1.870 -> 1.752 (-6.29% change)
Measured:          1.626 -> 1.643 (1.05% change)
Friction:          0.175 -> 0.145 (-17.30% change)

           LEXUS RX 2016 -> LEXUS RX HYBRID 2017
Factor:            1.588 -> 1.698 (6.98% change)
Measured:          1.043 -> 1.321 (26.70% change)
Friction:          0.213 -> 0.182 (-14.67% change)

       TOYOTA CAMRY 2018 -> TOYOTA CAMRY HYBRID 2018
Factor:            2.117 -> 1.996 (-5.71% change)
Measured:          1.716 -> 1.800 (4.90% change)
Friction:          0.105 -> 0.113 (7.01% change)

       TOYOTA CAMRY 2021 -> TOYOTA CAMRY HYBRID 2021
Factor:            2.446 -> 2.264 (-7.46% change)
Measured:          2.348 -> 2.390 (1.81% change)
Friction:          0.122 -> 0.115 (-5.23% change)

TOYOTA COROLLA TSS2 2019 -> TOYOTA COROLLA HYBRID TSS2 2019
Factor:            2.074 -> 1.908 (-8.02% change)
Measured:          1.926 -> 1.812 (-5.92% change)
Friction:          0

In [102]:
for hybrid_candidate, ice_candidate in hybrid_map.items():
  if hybrid_candidate in (CAR.CAMRYH, CAR.CHRH, CAR.CHRH_TSS2, CAR.ALPHARDH_TSS2):
    continue
  ecus = []
  if FW_VERSIONS[hybrid_candidate].keys() != FW_VERSIONS[ice_candidate].keys():
    missing = set(FW_VERSIONS[hybrid_candidate].keys()) ^ set(FW_VERSIONS[ice_candidate].keys())
    missing = {(ECU_NAME[i[0]], i[1], i[2]) for i in missing}
    print('ECU KEY DO NOT MATCH! Diff:', (hybrid_candidate, missing))
  

ECU KEY DO NOT MATCH! Diff: ('TOYOTA HIGHLANDER HYBRID 2018', {('engine', 1792, None), ('dsu', 1937, None), ('engine', 2016, None)})
ECU KEY DO NOT MATCH! Diff: ('LEXUS RX HYBRID 2017', {('engine', 1792, None), ('engine', 2016, None)})
ECU KEY DO NOT MATCH! Diff: ('TOYOTA COROLLA HYBRID TSS2 2019', {('engine', 2016, None)})
ECU KEY DO NOT MATCH! Diff: ('TOYOTA RAV4 HYBRID 2023', {('hybrid', 2002, None)})
ECU KEY DO NOT MATCH! Diff: ('LEXUS NX HYBRID 2018', {('engine', 1792, None), ('engine', 2016, None)})
ECU KEY DO NOT MATCH! Diff: ('LEXUS NX HYBRID 2020', {('engine', 1792, None), ('engine', 2016, None)})
ECU KEY DO NOT MATCH! Diff: ('LEXUS RX HYBRID 2020', {('engine', 1792, None), ('engine', 2016, None)})
